## Part 3

In [2]:
setwd("/home/leoKraushaar/Documents/School/Year 3/Semester 2/STAT 413/Project/protests/")
set.seed(42)

### Libraries

In [3]:
library(MASS)

### Clean Data

In [4]:
data <- read.csv("data/merged_data.csv")[, -1]
data$food <- NULL
data$manufac <- NULL

In [5]:
newMonth <- function(x) {
    if (x %in% c("December", "January", "February")) {
        return("Winter")
    } else if (x %in% c("March", "April", "May")) {
        return("Spring")
    } else if (x %in% c("June", "July", "August")) {
        return("Summer")
    } else {
        return("Fall")
    }
}

data$month <- sapply(data$month, newMonth)
colnames(data)[2] <- "season"

In [6]:
standardize <- function(x, mu, std) {
    return((x-mu)/std)
}

# data$pop <- sapply(data$pop, function(x) standardize(x, mean(data$pop), sd(data$pop)))

In [7]:
colnames(data)[colnames(data) == "GEO"] <- "prov"

data$prov  <- as.factor(data$prov)
data$season <- as.factor(data$season)
data$year <- as.factor(data$year)

In [8]:
data$retail <- sapply(data$retail, function(x) {standardize(x, mean(data$retail), sd(data$retail))})
data$power <- sapply(data$power, function(x) {standardize(x, mean(data$power), sd(data$power))})
# data$oil <- sapply(data$oil, function(x) {standardize(x, mean(data$oil), sd(data$oil))})

In [9]:
# data$retail <- NULL
head(data)

,year,season,prov,pop,protests,retail,oil,power
,<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>
1,2022,Spring,Alberta,4480956,17,0.4154217,3983,0.4393113
2,2022,Spring,British Columbia,5310164,42,0.5517610,77433,0.2824956
3,2022,Spring,Manitoba,1405197,2,-0.4144921,6290,-0.2989093
4,2022,Spring,New Brunswick,801778,5,-0.5188768,1818,-0.4874573
5,2022,Spring,Newfoundland and Labrador,529249,2,-0.5779367,77160,-0.5793902
6,2022,Spring,Northwest Territories,44828,0,-0.6965524,0,-0.6980796


In [10]:
cbind(data, interaction(data$oil, data$prov, labels=levels(data$prov)))

year,season,prov,pop,protests,retail,oil,power,"interaction(data$oil, data$prov, labels = levels(data$prov))"
<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<fct>
2022,Spring,Alberta,4480956,17,0.4154217,3983,0.4393113,3983.Alberta.Alberta
2022,Spring,British Columbia,5310164,42,0.5517610,77433,0.2824956,77433.British Columbia.British Columbia
2022,Spring,Manitoba,1405197,2,-0.4144921,6290,-0.2989093,6290.Manitoba.Manitoba
2022,Spring,New Brunswick,801778,5,-0.5188768,1818,-0.4874573,1818.New Brunswick.New Brunswick
2022,Spring,Newfoundland and Labrador,529249,2,-0.5779367,77160,-0.5793902,77160.Newfoundland and Labrador.Newfoundland and Labrador
2022,Spring,Northwest Territories,44828,0,-0.6965524,0,-0.6980796,0.Northwest Territories.Northwest Territories
2022,Spring,Nova Scotia,1014827,2,-0.4699081,47821,-0.5390880,47821.Nova Scotia.Nova Scotia
2022,Spring,Nunavut,40489,0,-0.7004528,0,-0.7061819,0.Nunavut.Nunavut
2022,Spring,Ontario,15046211,46,2.7521281,267687,1.3188851,267687.Ontario.Ontario


### Build Model

In [12]:
init_model <- glm.nb(protests ~ season + prov + retail/pop, data=data)

summary(step(init_model))

Start:  AIC=1579.35
protests ~ season + prov + retail/pop

             Df Deviance    AIC
<none>            351.68 1579.3
- retail:pop  1   357.47 1583.1
- season      3   401.03 1622.7
- prov       12   608.45 1812.1



Call:
glm.nb(formula = protests ~ season + prov + retail/pop, data = data, 
    init.theta = 8.865860964, link = log)

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                    4.490e+00  6.115e-01   7.342 2.11e-13 ***
seasonSpring                  -5.156e-02  8.130e-02  -0.634 0.525962    
seasonSummer                  -5.466e-01  8.495e-02  -6.435 1.24e-10 ***
seasonWinter                  -2.114e-01  8.734e-02  -2.421 0.015482 *  
provBritish Columbia           8.922e-01  1.634e-01   5.461 4.73e-08 ***
provManitoba                  -3.718e+00  1.196e+00  -3.109 0.001880 ** 
provNew Brunswick             -4.787e+00  1.390e+00  -3.444 0.000572 ***
provNewfoundland and Labrador -5.438e+00  1.501e+00  -3.624 0.000290 ***
provNorthwest Territories     -8.134e+00  1.733e+00  -4.693 2.69e-06 ***
provNova Scotia               -4.425e+00  1.310e+00  -3.378 0.000730 ***
provNunavut                   -7.771e+00  1.732e+00  -4.486 7.2

## Perform Monte Carlo Simulation